In [70]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_dark"

In [56]:
df = pd.read_csv('covid_data.csv')
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
0,16/05/2022,16,5,2022,3200.0,5.0,Austria,AT,AUT,8901064,Europe
1,15/05/2022,15,5,2022,2545.0,2.0,Austria,AT,AUT,8901064,Europe
2,14/05/2022,14,5,2022,3392.0,0.0,Austria,AT,AUT,8901064,Europe
3,13/05/2022,13,5,2022,3750.0,2.0,Austria,AT,AUT,8901064,Europe
4,12/05/2022,12,5,2022,4871.0,7.0,Austria,AT,AUT,8901064,Europe
...,...,...,...,...,...,...,...,...,...,...,...
24267,08/02/2020,8,2,2020,0.0,0.0,Sweden,SE,SWE,10327589,Europe
24268,07/02/2020,7,2,2020,0.0,0.0,Sweden,SE,SWE,10327589,Europe
24269,06/02/2020,6,2,2020,0.0,0.0,Sweden,SE,SWE,10327589,Europe
24270,05/02/2020,5,2,2020,0.0,0.0,Sweden,SE,SWE,10327589,Europe


In [57]:
df['dateRep'] = pd.to_datetime(df['dateRep'], infer_datetime_format=True)
df_filtered = df[df['dateRep'] >= '2020-02-20'].sort_values(by='dateRep')
df_filtered

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
24255,2020-02-20,20,2,2020,0.0,0.0,Sweden,SE,SWE,10327589,Europe
23393,2020-02-20,20,2,2020,53.0,0.0,Spain,ES,ESP,47332614,Europe
8145,2020-02-20,20,2,2020,0.0,0.0,France,FR,FRA,67320216,Europe
8988,2020-02-20,20,2,2020,2.0,1.0,Germany,DE,DEU,83166711,Europe
7277,2020-02-20,20,2,2020,0.0,0.0,Finland,FI,FIN,5525292,Europe
...,...,...,...,...,...,...,...,...,...,...,...
1618,2022-05-18,18,5,2022,NaN,6.0,Bulgaria,BG,BGR,6951482,Europe
10625,2022-05-18,18,5,2022,0.0,0.0,Iceland,IS,ISL,364134,Europe
4004,2022-05-18,18,5,2022,0.0,NaN,Czechia,CZ,CZE,10693939,Europe
9818,2022-05-18,18,5,2022,4749.0,103.0,Hungary,HU,HUN,9769526,Europe


In [62]:
df_world = df_filtered.groupby('dateRep').sum().reset_index()
df_world = df_world[['dateRep', 'cases', 'deaths']]
df_world['cases'] = df_world['cases'].astype(int)
df_world['deaths'] = df_world['deaths'].astype(int)
df_world

,dateRep,cases,deaths
0,2020-02-20,55,1
1,2020-02-21,60,0
2,2020-02-22,59,0
3,2020-02-23,73,0
4,2020-02-24,336,7
...,...,...,...
814,2022-05-14,117342,278
815,2022-05-15,77025,158
816,2022-05-16,126813,325
817,2022-05-17,202785,344


In [63]:
df_world['cumulated_cases'] = df_world['cases'].cumsum()
df_world['cumulated_deaths'] = df_world['deaths'].cumsum()

df_world.head(20)

,dateRep,cases,deaths,cumulated_cases,cumulated_deaths
0,2020-02-20,55,1,55,1
1,2020-02-21,60,0,115,1
2,2020-02-22,59,0,174,1
3,2020-02-23,73,0,247,1
4,2020-02-24,336,7,583,8
5,2020-02-25,227,3,810,11
6,2020-02-26,245,2,1055,13
7,2020-02-27,481,5,1536,18
8,2020-02-28,645,4,2181,22
9,2020-02-29,578,8,2759,30


In [65]:
df_world['rolling_cases'] = df_world['cases'].rolling(7).mean().round(0)
df_world['rolling_deaths'] = df_world['deaths'].rolling(7).mean().round(0)
df_world_filtered = df_world[df_world['dateRep'] >= '2020-03-01']
df_world_filtered_int = df_world_filtered.copy()
df_world_filtered_int[['rolling_cases', 'rolling_deaths']] = df_world_filtered[['rolling_cases', 'rolling_deaths']].astype(int)
df_world_filtered_int

,dateRep,cases,deaths,cumulated_cases,cumulated_deaths,rolling_cases,rolling_deaths
10,2020-03-01,1376,5,4135,35,555,5
11,2020-03-02,1289,18,5424,53,692,6
12,2020-03-03,1480,28,6904,81,871,10
13,2020-03-04,1938,32,8842,113,1112,14
14,2020-03-05,2609,43,11451,156,1416,20
...,...,...,...,...,...,...,...
814,2022-05-14,117342,278,141598970,1088603,189735,377
815,2022-05-15,77025,158,141675995,1088761,187290,365
816,2022-05-16,126813,325,141802808,1089086,183290,356
817,2022-05-17,202785,344,142005593,1089430,167568,338


In [66]:
df_world_filtered_int.head(20)

,dateRep,cases,deaths,cumulated_cases,cumulated_deaths,rolling_cases,rolling_deaths
10,2020-03-01,1376,5,4135,35,555,5
11,2020-03-02,1289,18,5424,53,692,6
12,2020-03-03,1480,28,6904,81,871,10
13,2020-03-04,1938,32,8842,113,1112,14
14,2020-03-05,2609,43,11451,156,1416,20
15,2020-03-06,3090,57,14541,213,1766,27
16,2020-03-07,3812,45,18353,258,2228,33
17,2020-03-08,4313,149,22666,407,2647,53
18,2020-03-09,6616,123,29282,530,3408,68
19,2020-03-10,7602,201,36884,731,4283,93


In [75]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=df_world_filtered_int['dateRep'],
    y=df_world_filtered_int['cumulated_cases'],
    name='Cases',
    mode='lines'
), secondary_y=False)

fig.add_trace(go.Scatter(
    x=df_world_filtered_int['dateRep'],
    y=df_world_filtered_int['cumulated_deaths'],
    name='Deaths',
    mode='lines'
), secondary_y=True)

fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Cases", secondary_y=False)
fig.update_yaxes(title_text="Deaths", secondary_y=True)

fig.update_layout(width=1200, height=600, title='Cumulated Cases and Deaths in the World', title_x=0.5)
fig.show()

In [79]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_world_filtered_int['dateRep'],
    y=df_world_filtered_int['cases'],
    name='Cases per Day',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=df_world_filtered_int['dateRep'],
    y=df_world_filtered_int['rolling_cases'],
    name='7-day Rolling Average',
    mode='lines'
))

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cases per Day")

fig.update_layout(width=1200, height=600, title='New Cases in the World', title_x=0.5)
fig.show()


In [80]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_world_filtered_int['dateRep'],
    y=df_world_filtered_int['deaths'],
    name='Deaths per Day',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=df_world_filtered_int['dateRep'],
    y=df_world_filtered_int['rolling_deaths'],
    name='7-day Rolling Average',
    mode='lines'
))

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Deaths per Day")

fig.update_layout(width=1200, height=600, title='New Deaths in the World', title_x=0.5)
fig.show()

In [87]:
df_country = df_filtered[df_filtered['countriesAndTerritories'] == 'France'].sort_values(by='dateRep')[['dateRep', 'countriesAndTerritories', 'cases', 'deaths']]
df_country[['cases', 'deaths']] = df_country[['cases', 'deaths']].astype(int)
df_country['rolling_cases'] = df_country['cases'].rolling(7).mean().round(0)
df_country['rolling_deaths'] = df_country['deaths'].rolling(7).mean().round(0)
df_country = df_country[df_country['dateRep'] >= '2020-03-01']
df_country.reset_index(drop=True, inplace=True)
df_country

,dateRep,countriesAndTerritories,cases,deaths,rolling_cases,rolling_deaths
0,2020-03-01,France,45,0,19.0,0.0
1,2020-03-02,France,46,0,26.0,0.0
2,2020-03-03,France,21,0,28.0,0.0
3,2020-03-04,France,73,0,38.0,0.0
4,2020-03-05,France,138,0,55.0,0.0
...,...,...,...,...,...,...
803,2022-05-13,France,32773,98,34315.0,93.0
804,2022-05-14,France,30459,74,33244.0,94.0
805,2022-05-15,France,22844,21,32318.0,90.0
806,2022-05-16,France,5936,126,32115.0,89.0


In [98]:
growth_cases = (df_country.iloc[-1]['rolling_cases'] - df_country.iloc[-2]['rolling_cases']) / df_country.iloc[-2]['rolling_cases'] * 100
growth_deaths = (df_country.iloc[-1]['rolling_deaths'] - df_country.iloc[-2]['rolling_deaths']) / df_country.iloc[-2]['rolling_deaths'] * 100
country = df_country.iloc[-1]['countriesAndTerritories']
country

'France'

In [101]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_country['dateRep'],
    y=df_country['cases'],
    name='Cases per Day',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=df_country['dateRep'],
    y=df_country['rolling_cases'],
    name='7-day Rolling Average',
    mode='lines'
))

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cases per Day")

fig.update_layout(width=1200, height=600, title=f'New Cases in {country} per Day', title_x=0.5)
fig.show()

In [102]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_country['dateRep'],
    y=df_country['deaths'],
    name='Deaths per Day',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=df_country['dateRep'],
    y=df_country['rolling_deaths'],
    name='7-day Rolling Average',
    mode='lines'
))

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Deaths per Day")

fig.update_layout(width=1200, height=600, title=f'New Deaths in {country} per Day', title_x=0.5)
fig.show()